In [1]:
import pandas as pd 
import requests 
import json 
import matplotlib.pyplot as plt
import numpy as np 

# make a GET request to the endpoint and store the response in a variable called response
response = requests.get("https://min-api.cryptocompare.com/data/histohour?fsym=BTC&tsym=USD&e=Kraken&limit=2000&aggregate=1&timestamp='2017-04-01'")
bitcoin_dict = response.json()
columns = ["close","high","low","open","time","volumefrom","volumeto"]

In [2]:
# Generate a dataframe named bitcoin_data with ohlcv data in 1 hour 
bitcoin_data = pd.DataFrame(bitcoin_dict["Data"],columns=columns)


In [3]:
# Create the column Date that convert the time column in a timestamp object 
bitcoin_data["Date"] = pd.to_datetime(bitcoin_data["time"], unit="s")
bitcoin_data = bitcoin_data.drop("time",axis=1)



In [4]:
bitcoin_data = bitcoin_data[["Date","open","high","low","close","volumefrom","volumeto"]]

In [5]:
# Calculate the twenty day simple moving average and the fifty day simply moving average. These will be the indicators of the strategy
bitcoin_data["20d"] = np.round(bitcoin_data["close"].rolling(window = 20, center = False).mean(), 2)
bitcoin_data["50d"] = np.round(bitcoin_data["close"].rolling(window = 50, center = False).mean(), 2)


bitcoin_data.tail(10)


,Date,open,high,low,close,volumefrom,volumeto,20d,50d
1991,2017-07-30 05:00:00,2720.90,2720.90,2668.34,2674.00,512.26,1380583.77,2712.11,2745.20
1992,2017-07-30 06:00:00,2674.00,2697.00,2653.00,2680.00,593.59,1587190.86,2710.15,2745.28
1993,2017-07-30 07:00:00,2680.00,2695.00,2656.50,2672.60,264.62,707809.12,2709.28,2744.65
1994,2017-07-30 08:00:00,2672.60,2689.80,2665.11,2674.96,116.64,311439.00,2707.82,2744.21
1995,2017-07-30 09:00:00,2674.96,2709.00,2669.51,2675.00,148.17,398689.73,2707.17,2742.77
1996,2017-07-30 10:00:00,2675.00,2698.47,2670.00,2671.00,66.09,177093.44,2705.58,2740.69
1997,2017-07-30 11:00:00,2671.00,2694.50,2671.00,2692.01,67.70,181724.33,2703.79,2739.03
1998,2017-07-30 12:00:00,2692.01,2710.85,2681.54,2692.63,75.73,204084.94,2703.41,2737.61
1999,2017-07-30 13:00:00,2692.63,2703.49,2683.80,2700.00,81.98,220647.06,2702.11,2734.69
2000,2017-07-30 14:00:00,2700.00,2748.22,2689.00,2735.73,157.96,429859.61,2701.21,2733.22


In [6]:
bitcoin_data["H-L"] = bitcoin_data["high"] - bitcoin_data["low"]
bitcoin_data["H-YC"] = abs(bitcoin_data["high"] - bitcoin_data["close"].shift(1))
bitcoin_data["L-YC"] = abs(bitcoin_data["low"] - bitcoin_data["close"].shift(1))

In [7]:
def backTesting():
    x = 0
    count=0
    lastBoughtFor = 0
    stance = "none"
    totalProfit = 0
     #datetime.datetime.fromtimestamp(float(bitcoin_OHLCV.index)).strftime("%Y-%m-%d")))
    while x < len(bitcoin_data):
        if stance == "none":
            if (bitcoin_data["20d"][x] > bitcoin_data["50d"][x]): #& (bitcoin_data["ADX"][x] >20): #& (bitcoin_data["close"][x] >bitcoin_data["20d"][x]): # oversold, let´s buy
                print("starting time", str(bitcoin_data["Date"][x]))
                stance = "holding"
                print("Buying BTC @", bitcoin_data["close"][x])
                lastBoughtFor = bitcoin_data["close"][x]
                
        elif stance == "holding":
            if bitcoin_data["20d"][x] < bitcoin_data["50d"][x]:
                stance = "none"
                print("Selling BTC @", bitcoin_data["close"][x])
                fees = (.002)*(bitcoin_data["close"][x] + lastBoughtFor)
                print("Fee for this was",fees)
                print("finish time", str(bitcoin_data["Date"][x]))
                print("Profit on this trade:",bitcoin_data["close"][x] - lastBoughtFor - fees) # if we lost money this is negative 
                totalProfit += (bitcoin_data["close"][x] - lastBoughtFor - fees)
                count +=1
            
        
        x +=1
    print("total profit of", totalProfit)
    print(count)
backTesting()

starting time 2017-05-10 07:00:00
Buying BTC @ 1725.12
Selling BTC @ 1768.0
Fee for this was 6.98624
finish time 2017-05-12 13:00:00
Profit on this trade: 35.89376
starting time 2017-05-14 03:00:00
Buying BTC @ 1795.0
Selling BTC @ 1738.0
Fee for this was 7.066
finish time 2017-05-15 10:00:00
Profit on this trade: -64.066
starting time 2017-05-16 22:00:00
Buying BTC @ 1778.4
Selling BTC @ 2560.01
Fee for this was 8.67682
finish time 2017-05-26 08:00:00
Profit on this trade: 772.93318
starting time 2017-05-28 15:00:00
Buying BTC @ 2095.0
Selling BTC @ 2178.93
Fee for this was 8.54786
finish time 2017-05-31 05:00:00
Profit on this trade: 75.38214
starting time 2017-05-31 23:00:00
Buying BTC @ 2291.3
Selling BTC @ 2676.79
Fee for this was 9.93618
finish time 2017-06-07 21:00:00
Profit on this trade: 375.55382
starting time 2017-06-09 07:00:00
Buying BTC @ 2811.77
Selling BTC @ 2836.06
Fee for this was 11.29566
finish time 2017-06-11 09:00:00
Profit on this trade: 12.99434
starting time 20

In [8]:
# In this cell I calculate the entry and exit signals of the strategy. When the fast moving average, cross up the slow moving average, an entry signal is triger. 
# The entry_signals variable take the value of 1 if this set up happened ans zero otherwise.
bitcoin_data["entry_signals"] = np.where((bitcoin_data["20d"] > bitcoin_data["50d"]) & (bitcoin_data["20d"].shift(1) < bitcoin_data["50d"].shift(1)),1,0)
bitcoin_data["exit_signals"] = np.where((bitcoin_data["20d"] < bitcoin_data["50d"]),1,0)

In [9]:
bitcoin_data.tail()

,Date,open,high,low,close,volumefrom,volumeto,20d,50d,H-L,H-YC,L-YC,entry_signals,exit_signals
1996,2017-07-30 10:00:00,2675.00,2698.47,2670.00,2671.00,66.09,177093.44,2705.58,2740.69,28.47,23.47,5.00,0,1
1997,2017-07-30 11:00:00,2671.00,2694.50,2671.00,2692.01,67.70,181724.33,2703.79,2739.03,23.50,23.50,0.00,0,1
1998,2017-07-30 12:00:00,2692.01,2710.85,2681.54,2692.63,75.73,204084.94,2703.41,2737.61,29.31,18.84,10.47,0,1
1999,2017-07-30 13:00:00,2692.63,2703.49,2683.80,2700.00,81.98,220647.06,2702.11,2734.69,19.69,10.86,8.83,0,1
2000,2017-07-30 14:00:00,2700.00,2748.22,2689.00,2735.73,157.96,429859.61,2701.21,2733.22,59.22,48.22,11.00,0,1


In [10]:
# This lines yield that there are a lof of exit_signals. We have to filtered the exit_signals variable in order to utilize this
# variable if and only if there is a trade in place. 
print(len(bitcoin_data[bitcoin_data["entry_signals"] ==1]))
print(len(bitcoin_data[bitcoin_data["exit_signals"]==1]))

18
793


In [11]:
# Create a series pandas filtered_exit with nan values, to populate afterwards with the logic of the order management.
bitcoin_data["filtered_exit"] = np.nan

In [12]:
for i, data in bitcoin_data.iterrows():
    holding_status = (bitcoin_data.loc[:(i-1), 'entry_signals'] - bitcoin_data.loc[:(i-1), 'filtered_exit']).sum()
    
    if bitcoin_data["exit_signals"][i] == 0 or i == 0:
        bitcoin_data.loc[i, 'filtered_exit'] = 0
    elif bitcoin_data["exit_signals"][i] == 1:
        if holding_status == 0:
            bitcoin_data.loc[i, 'filtered_exit'] = 0
        elif holding_status == 1:
            bitcoin_data.loc[i, 'filtered_exit'] = 1

In [13]:
# With the for loop before, we get an exit signal for each of the entry signals, and therefore we get all the accurate signals within the strategy
print(len(bitcoin_data[bitcoin_data["entry_signals"] ==1]))
print(len(bitcoin_data[bitcoin_data["filtered_exit"]==1]))

18
18


In [14]:
# Create the trades data frame with some information of the trades in the strategy 

trades = pd.concat([pd.DataFrame({"Date": bitcoin_data.loc[bitcoin_data["entry_signals"]==1,"Date"], "Price": bitcoin_data.loc[bitcoin_data["entry_signals"]==1,"close"],
                                   "Regime":bitcoin_data.loc[bitcoin_data["entry_signals"]==1,"entry_signals"],
                                   "Signal": "Buy"}),
                    pd.DataFrame({"Date": bitcoin_data.loc[bitcoin_data["filtered_exit"]==1,"Date"], "Price":bitcoin_data.loc[bitcoin_data["filtered_exit"]==1,"close"],
                                 "Regime":bitcoin_data.loc[bitcoin_data["filtered_exit"]==1,"filtered_exit"],
                                 "Signal":"Sell"}),])
trades.sort_index(inplace=True)
trades

,Date,Price,Regime,Signal
141,2017-05-14 03:00:00,1795.00,1.0,Buy
172,2017-05-15 10:00:00,1738.00,1.0,Sell
208,2017-05-16 22:00:00,1778.40,1.0,Buy
434,2017-05-26 08:00:00,2560.01,1.0,Sell
489,2017-05-28 15:00:00,2095.00,1.0,Buy
551,2017-05-31 05:00:00,2178.93,1.0,Sell
569,2017-05-31 23:00:00,2291.30,1.0,Buy
735,2017-06-07 21:00:00,2676.79,1.0,Sell
769,2017-06-09 07:00:00,2811.77,1.0,Buy
819,2017-06-11 09:00:00,2836.06,1.0,Sell


In [15]:
# Make and index timestamp with the Date Column of the trades dataframe
trades.index = pd.to_datetime(trades["Date"],unit="s")
trades = trades.drop("Date",axis=1)


In [16]:
# Create the trades_long_profits that show the datetime at the entry moment(index) of the trade, the end time of the trade, the price at which
# the trades was trigered and the profit of the trade
trades_long_profits = pd.DataFrame({
        
    "End_date":  trades["Price"].loc[trades.loc[trades["Signal"].shift(1)=="Buy"].index].index,
    "Price":trades.loc[(trades["Signal"]=="Buy") & trades["Regime"]==1,"Price"],
    "Profit":pd.Series(trades["Price"] - trades["Price"].shift(1) - (0.002*(trades["Price"] + trades["Price"].shift(1)))).loc[trades.loc[(trades["Signal"].shift(1)=="Buy")
        &(trades["Regime"].shift(1)==1)].index].tolist(),
                       })
trades_long_profits

,End_date,Price,Profit
Date,,,
2017-05-14 03:00:00,2017-05-15 10:00:00,1795.00,-64.06600
2017-05-16 22:00:00,2017-05-26 08:00:00,1778.40,772.93318
2017-05-28 15:00:00,2017-05-31 05:00:00,2095.00,75.38214
2017-05-31 23:00:00,2017-06-07 21:00:00,2291.30,375.55382
2017-06-09 07:00:00,2017-06-11 09:00:00,2811.77,12.99434
2017-06-11 19:00:00,2017-06-12 16:00:00,2884.00,-321.72438
2017-06-14 03:00:00,2017-06-14 21:00:00,2752.59,-312.99518
2017-06-16 14:00:00,2017-06-18 23:00:00,2459.20,28.64608
2017-06-20 03:00:00,2017-06-22 04:00:00,2600.00,-18.71334


In [17]:
investment = 100000
trades_long_profits["portfolio_value"] =  investment  + trades_long_profits["Profit"].cumsum()
trades_long_profits["returns"] = ((trades_long_profits["portfolio_value"] / trades_long_profits["portfolio_value"].shift(1)) -1) * 100


In [18]:
import math
def create_drawdown(pnl):
    hwm =[0]
    idx = pnl.index
    drawdown = pd.Series(index = idx)
    duration = pd.Series(index = idx)
    for t in range(1, len(idx)):
        hwm.append(max(hwm[t-1], pnl[t]))
        drawdown[t] = (hwm[t] -pnl[t]) 
        duration[t] = (0 if drawdown[t] == 0 else duration[t-1] +1)
    return drawdown, drawdown.max() , duration.max() 

trades_long_profits["drawdown"] = create_drawdown(trades_long_profits["returns"])[0]
trades_long_profits["net_profit"] = trades_long_profits["portfolio_value"] - 100000
trades_long_profits

,End_date,Price,Profit,portfolio_value,returns,drawdown,net_profit
Date,,,,,,,
2017-05-14 03:00:00,2017-05-15 10:00:00,1795.00,-64.06600,99935.93400,NaN,NaN,-64.06600
2017-05-16 22:00:00,2017-05-26 08:00:00,1778.40,772.93318,100708.86718,0.773429,0.000000,708.86718
2017-05-28 15:00:00,2017-05-31 05:00:00,2095.00,75.38214,100784.24932,0.074852,0.698577,784.24932
2017-05-31 23:00:00,2017-06-07 21:00:00,2291.30,375.55382,101159.80314,0.372631,0.400797,1159.80314
2017-06-09 07:00:00,2017-06-11 09:00:00,2811.77,12.99434,101172.79748,0.012845,0.760583,1172.79748
2017-06-11 19:00:00,2017-06-12 16:00:00,2884.00,-321.72438,100851.07310,-0.317995,1.091424,851.07310
2017-06-14 03:00:00,2017-06-14 21:00:00,2752.59,-312.99518,100538.07792,-0.310354,1.083783,538.07792
2017-06-16 14:00:00,2017-06-18 23:00:00,2459.20,28.64608,100566.72400,0.028493,0.744936,566.72400
2017-06-20 03:00:00,2017-06-22 04:00:00,2600.00,-18.71334,100548.01066,-0.018608,0.792037,548.01066


In [19]:
Total_Gross_Win = trades_long_profits[trades_long_profits["Profit"] > 0].sum()["Profit"]
Total_Gross_Loss = trades_long_profits[trades_long_profits["Profit"] < 0].sum()["Profit"]
Profit_Factor = abs(Total_Gross_Win / Total_Gross_Loss)
Total_Net_Profit = (Total_Gross_Win + Total_Gross_Loss)
Number_Winning_Trades = trades_long_profits[trades_long_profits["Profit"] > 0].count()["Profit"]
Number_Lossing_Trades = trades_long_profits[trades_long_profits["Profit"] < 0].count()["Profit"]
Percentage_Profitable_Trades = (Number_Winning_Trades) / (Number_Winning_Trades + Number_Lossing_Trades)
Largest_Profitable_Trade = max(trades_long_profits["Profit"])
Largest_Losing_Trade = min(trades_long_profits["Profit"])
Max_Draw_Down = trades_long_profits["drawdown"].max()

In [20]:
def sharpe_ratio(returns):
    return (returns.mean() / returns.std()) * np.sqrt(len(trades_long_profits))
Sharpe_ratio = sharpe_ratio(trades_long_profits["returns"])

In [21]:
print("Total_Gross_Win:", Total_Gross_Win )
print("Total_Gross_Loss:", Total_Gross_Loss)
print("Profit_Factor:", Profit_Factor)
print("Total_Net_Profit", Total_Net_Profit)
print("Number_Winning_Trades:", Number_Winning_Trades)
print("Average_Winning_Trade:",Total_Gross_Win / Number_Winning_Trades)
print("Number_Lossing_Trades:", Number_Lossing_Trades)
print("Average_Lossing_Trade:", Total_Gross_Loss / Number_Lossing_Trades)
print("Percentage_Profitable_Trades:", Percentage_Profitable_Trades)
print("Largest_Profitable_Trade:", Largest_Profitable_Trade)
print("Largest_Losing_Trade:", Largest_Losing_Trade)
print("Max_Draw_Down:", Max_Draw_Down)
print("Sharpe_ratio:",Sharpe_ratio)

Total_Gross_Win: 1843.2696
Total_Gross_Loss: -938.73238
Profit_Factor: 1.9635730473
Total_Net_Profit 904.53722
Number_Winning_Trades: 10
Average_Winning_Trade: 184.32696
Number_Lossing_Trades: 8
Average_Lossing_Trade: -117.3415475
Percentage_Profitable_Trades: 0.555555555556
Largest_Profitable_Trade: 772.93318
Largest_Losing_Trade: -321.72438
Max_Draw_Down: 1.09142362814
Sharpe_ratio: 0.964707057431


The strategy shows good result with a Total Net Profit of U$S 1234. With this strategy we will win 6 times out of 16 times, but the average wining trade(U$S317) is much higher than the average lossing trades(U$S-67)

The sharpe ratio is satisfactory and also it is possible to optimize the strategy by apply some risk management techniques that increment or diminish the size of the trade in behalf the market condition. 
The only matter of this strategy is that at some point we have to bear with an important drawdawn, there are some lossing trades of U$S-162, and U$S-138.